In [1]:
import numpy as np
import torch as th
import torch.nn as nn
import os
import json
import pandas as pd

from train.multi_hyperparameter import MultiHyperparameter
from custom_models.unet_original import UNet, UNet2
from train.unet_trainer import UnetTrainer
from datasets.semantic_dataset import SemanticDataset

C:\Users\silva\anaconda3\envs\TorchEnv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'blocks'

In [3]:
params = {'padding_mode': ['zeros', 'reflect', 'replicate', 'circular'],
          'depth': 5,
          'start_layers': 64,
          'dim_multiplier': 2,
          'input_conv_kernel_size': 3,
          'out_classes': 2,
          'criterion': nn.CrossEntropyLoss(),
          'optimizer': 1}

param = {'padding_mode': 'zeros',
          'depth': 5,
          'start_layers': 64,
          'dim_multiplier': 2,
          'input_conv_kernel_size': 3,
          'out_classes': 2,
          'criterion': nn.CrossEntropyLoss(),
          'optimizer': 1}

unet_hyps = MultiHyperparameter(params)
unet = UNet2.__new__(UNet2)

models_to_evaluate = [(unet, unet_hyps)]

num_epochs = 1
models = []
trainers = []
folds = [0, 1, 2, 3]
device = th.device("cuda" if th.cuda.is_available() else "cpu")
cwd = os.getcwd()
ds1_path = os.path.join(cwd, '../data/cell_type_1')
ds2_path = os.path.join(cwd, '../data/cell_type_2')
result_path = os.path.join(cwd, '../results')

class CrossTrainEvaluator:
    def __init__(self, model, datasets_path, results_path):
        self.model = model
        self.datasets_path = datasets_path
        #self.params_to_test = params_to_test
        self.results_path = results_path
        self.folds = [0, 1, 2, 3]
        self.evaluated = []

    def evaluate_param(self, param, folds, epochs, test=False):
        print(param)
        current_model = self.model
        report = pd.DataFrame()

        comb_train_losses = np.zeros(num_epochs)
        comb_validation_losses = np.zeros(num_epochs)

        for i, dataset_path in enumerate(self.datasets_path, 0):
            train_losses = np.zeros(num_epochs)
            validation_losses = np.zeros(num_epochs)
            for train_fold in folds:
                folds_train = folds.copy()
                folds_train.remove(train_fold)
                folds_validate = [train_fold]
                current_model.__init__(param)
                ds_train = SemanticDataset(dataset_path, folds_train)
                ds_validate = SemanticDataset(dataset_path, folds_validate)
                trainer = UnetTrainer(current_model, device, param['criterion'],
                                      th.optim.Adam(current_model.parameters(), lr=1e-4, weight_decay=1e-5), ds_train, ds_validate)
                trainer.train(epochs, test=True)
                train_loss, validation_loss = trainer.get_losses()
                train_losses += train_loss
                validation_losses += validation_loss

            train_losses /= len(folds)
            validation_losses /= len(folds)
            comb_train_losses += train_losses
            comb_validation_losses += validation_losses

            key = 'ds_' + str(i)
            report[key + '_train'] = train_losses
            report[key + '_validate'] = validation_losses

        comb_train_losses /= 2
        comb_validation_losses /= 2
        report['combined_train'] = comb_train_losses
        report['combined_validate'] = comb_validation_losses

        #report.to_csv(self.results_path + '/test.csv', index=False)
        th.save({'model': self.model, 'params': params, 'report': report}, self.results_path + '/test.pt')

        return report

cte = CrossTrainEvaluator(unet, [ds1_path, ds2_path], result_path)
report = cte.evaluate_param(param, [0, 1, 2, 3], 1)
print(report)
# (self, model, datasets_path, params_to_test: MultiHyperparameter, results_path, epochs):
# (self, param, model, folds, epochs):

{'padding_mode': 'zeros', 'out_classes': 2, 'criterion': CrossEntropyLoss(), 'optimizer': 1}
   ds_0_train  ds_0_validate  ds_1_train  ds_1_validate  combined_train  \
0    0.396079       0.304049     0.39983       0.293947        0.397955   

   combined_validate  
0           0.298998  


In [6]:
file = th.load(os.path.join(cwd, '../results/test.pt'))
print(file[0]['criterion'])

CrossEntropyLoss()


In [19]:
    def cross_validate(self, epochs, mode='grid', num_params=0):
        # assert (mode == 'random' and num_params > 0) or mode == 'grid'
        # if mode == 'grid':
        #     params_to_evaluate = self.params_to_test.get_full_grid_params()
        # else:
        #     params_to_evaluate = self.params_to_test.get_random_params(num_params)
        #
        # for fold in self.folds:
        #     folds_ds_train = folds.copy()
        #     folds_ds_train.remove(fold)
        #     folds_ds_test = [fold]
        #
        #     for param in params_to_evaluate:
        #         print(param)
        #         report = {'param': param}
        #
        #         comb_train_losses = np.zeros(num_epochs)
        #         comb_validation_losses = np.zeros(num_epochs)
        #
        #         for i, dataset_path in enumerate(self.datasets_path, 0):
        #             folds_train = folds_ds_train.copy()
        #             folds_train.remove(train_fold)
        #             folds_validate = [train_fold]
        #             train_losses = np.zeros(num_epochs)
        #             validation_losses = np.zeros(num_epochs)
        #             for train_fold in folds_ds_train:
        #                 model.__init__(param)
        #                 ds_train = SemanticDataset(dataset_path, folds_train)
        #                 ds_validate = SemanticDataset(dataset_path, folds_validate)
        #                 trainer = UnetTrainer(model, device, param['criterion'],
        #                                       th.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5), ds_train, ds_validate)
        #                 trainer.train(self.num_epochs, test=True)
        #                 train_loss, validation_loss = trainer.get_losses()
        #                 train_losses += train_loss
        #                 validation_losses += validation_loss
        #
        #             train_losses /= len(folds_ds_train)
        #             validation_losses /= len(folds_ds_train)
        #             comb_train_losses += train_losses
        #             comb_validation_losses += validation_losses
        #
        #             key = 'ds_' + str(i)
        #             report[key] = {'train': train_losses, 'validate': validation_losses}
        #
        #         comb_train_losses /= 2
        #         comb_validation_losses /= 2
        #         report['combined'] = {'train': train_losses, 'validate': validation_losses}
        #
        #     model.__init__(param)
        #     ds_train = SemanticDataset(dataset_path, folds_train)
        #     ds_validate = SemanticDataset(dataset_path, folds_validate)
        #     trainer = UnetTrainer(model, device, param['criterion'],
        #                           th.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5), ds_train, ds_validate)
        #     trainer.train(self.num_epochs, test=True)
        #     train_loss, validation_loss = trainer.get_losses()
        #     train_losses += train_loss
        #     validation_losses += validation_loss